<a href="https://colab.research.google.com/github/wasiq921/Task-3-ADVANCED-LEVEL--LGM-VIP-AUGUST-2021/blob/main/Task_2_(ADVANCED_LEVEL_TASK)_LGM_VIP_AUGUST_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 2 Next Word Prediction

### Using Tensorflow and Keras library train a RNN , to predict the next word. 
#### Dataset Link: https://drive.google.com/file/d/1GeUzNVqiixXHnTl8oNiQ2W3CynX_lsu2/view

## Author: Muhammad Wasiq
##LGM VIP AUGUST 2021


In [ ]:
#import all the required libraries we need for this task
import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import heapq

In [ ]:
#loading the Dataset
path = '1661-0.txt'
text = open(path).read().lower()
print('corpus length:', len(text))

corpus length: 581888


In [ ]:
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)

In [ ]:
words

['project',
 'gutenberg',
 's',
 'the',
 'adventures',
 'of',
 'sherlock',
 'holmes',
 'by',
 'arthur',
 'conan',
 'doyle',
 'this',
 'ebook',
 'is',
 'for',
 'the',
 'use',
 'of',
 'anyone',
 'anywhere',
 'at',
 'no',
 'cost',
 'and',
 'with',
 'almost',
 'no',
 'restrictions',
 'whatsoever',
 'you',
 'may',
 'copy',
 'it',
 'give',
 'it',
 'away',
 'or',
 're',
 'use',
 'it',
 'under',
 'the',
 'terms',
 'of',
 'the',
 'project',
 'gutenberg',
 'license',
 'included',
 'with',
 'this',
 'ebook',
 'or',
 'online',
 'at',
 'www',
 'gutenberg',
 'net',
 'title',
 'the',
 'adventures',
 'of',
 'sherlock',
 'holmes',
 'author',
 'arthur',
 'conan',
 'doyle',
 'release',
 'date',
 'november',
 '29',
 '2002',
 'ebook',
 '1661',
 'last',
 'updated',
 'may',
 '20',
 '2019',
 'language',
 'english',
 'character',
 'set',
 'encoding',
 'utf',
 '8',
 'start',
 'of',
 'this',
 'project',
 'gutenberg',
 'ebook',
 'the',
 'adventures',
 'of',
 'sherlock',
 'holmes',
 'produced',
 'by',
 'an',
 'ano

In [ ]:
unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

In [ ]:
#Feature Engineering

WORD_LENGTH = 5
prev_words = []
next_words = []
for i in range(len(words) - WORD_LENGTH):
    prev_words.append(words[i:i + WORD_LENGTH])
    next_words.append(words[i + WORD_LENGTH])
print(prev_words[0])
print(next_words[0])

['project', 'gutenberg', 's', 'the', 'adventures']
of


In [ ]:
#We will now create 2 NumPy Arrays, x for storing the features and y for storing its corresponding label. if the word is available both X and Y becomes 1
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next_words[i]]] = 1

In [ ]:
print(X[0][0])

[False False False ... False False False]


##Model Building - RNN

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               4264960   
_________________________________________________________________
dense (Dense)                (None, 8201)              1057929   
_________________________________________________________________
activation (Activation)      (None, 8201)              0         
Total params: 5,322,889
Trainable params: 5,322,889
Non-trainable params: 0
_________________________________________________________________


## Model Training

In [ ]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=20, shuffle=True).history

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/20
811/811 [==============================] - 290s 355ms/step - loss: 5.7957 - accuracy: 0.1750 - val_loss: 8.0124 - val_accuracy: 0.1097
Epoch 2/20
811/811 [==============================] - 284s 350ms/step - loss: 5.4209 - accuracy: 0.2115 - val_loss: 8.2326 - val_accuracy: 0.1109
Epoch 3/20
811/811 [==============================] - 285s 351ms/step - loss: 5.0820 - accuracy: 0.2510 - val_loss: 8.3132 - val_accuracy: 0.1007
Epoch 4/20
811/811 [==============================] - 278s 343ms/step - loss: 4.8100 - accuracy: 0.2895 - val_loss: 8.4893 - val_accuracy: 0.0879
Epoch 5/20
811/811 [==============================] - 281s 347ms/step - loss: 4.5698 - accuracy: 0.3285 - val_loss: 8.8796 - val_accuracy: 0.0882
Epoch 6/20
811/811 [==============================] - 278s 343ms/step - loss: 4.3484 - accuracy: 0.3687 - val_loss: 8.7436 - val_accuracy: 0.0817
Epoch 7/20
811/811 [==============================] - 278s 342ms/step - loss: 4.1733 - accuracy: 0.4040 - val_loss: 9.1702 -

In [ ]:
#Sucessfully trained our model, now we are saving it
model.save('keras_next_word_model.h5')

model = load_model('keras_next_word_model.h5')


#Testing Next Word Prediction Model

In [ ]:
def prepare_input(text):
    x = np.zeros((1, WORD_LENGTH, len(unique_words)))
    for t, word in enumerate(text.split()):
        print(word)
        x[0, t, unique_word_index[word]] = 1
    return x
prepare_input("It is not a lack".lower())


it
is
not
a
lack


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [ ]:

unique_words[3010]

'for'

In [ ]:
txt = input('Enter text :')

prepare_input(txt.lower())

Enter text :he turned hungrily on
he
turned
hungrily
on


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [ ]:
txt.split()[-4:]

['he', 'turned', 'hungrily', 'on']

In [ ]:
def pediction(model,tokenize,txt):
    sequence1 = tokenize.tokenize(txt)
    sequence = np.array(sequence1)
    print(sequence1)
    preds= np.argmax(model.predict(sequence1))
    pred_wods=""
    print(preds)

In [ ]:
prepare_input('she Sherlock Holmes she'.lower())

she
sherlock
holmes
she


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [ ]:
unique_words[11]

'12_s_'